In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import sys
import shutil

import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from itertools import combinations

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
import seaborn as sns
from random import sample, seed

In [2]:
working_dir = 'tmp/'

# File inputs
# x_input = 'data/ProCan-DepMapSanger_protein_matrix_6692_averaged.xlsx'
x_input = 'data/train_protein_matrix.csv'
y_input = 'data/DrugResponse_PANCANCER_GDSC1_GDSC2_20200602.csv'

test_input = 'data/test_protein_matrix.csv'


In [3]:
x = pd.read_csv(x_input)
c = [a.replace('.','').replace('_','') for a in x.columns]
x.columns = c

x_test = pd.read_csv(test_input)
x_test.columns = c

y = pd.read_csv(y_input)[['drug_id','cell_line_name','ln_IC50', 'max_screening_conc']]

C:\Users\wen068\AppData\Local\Temp\ipykernel_19028\2547150521.py:8: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  y = pd.read_csv(y_input)[['drug_id','cell_line_name','ln_IC50', 'max_screening_conc']]


In [4]:
drug_list = np.unique(y.drug_id)
protein_list = c[1:]

In [13]:
def results_fit_to_df(results, ols, y, test_data):
    coeffs = results.params.tolist()
    pvals = results.pvalues.tolist()
    pseudo_r2 = results.rsquared
    # adj_r2 = results.rsquared_adj
    tvals = results.tvalues.tolist()
    cint_low = results.conf_int()[0].tolist()
    cint_high = results.conf_int()[1].tolist()

    pred_y = results.predict()
    train_pear_R = pearsonr(pred_y, y).statistic
    train_mse = np.square(y - pred_y).mean()

    if len(test_data.ln_IC50) < 8:
        test_pear_R = None
        test_mse = None
    else:    
        test_y = test_data.ln_IC50.to_list()
        pred_y = results.predict(test_data)
        test_pear_R = pearsonr(pred_y, test_y).statistic   # want 20 entries for this
        test_mse = np.square(test_y - pred_y).mean()

    
    try:
        results = results.summary()
    except:
        #ValueError: resids must contain at least 2 elements
        r = pd.DataFrame([1,2,3]) #dirty...
        r['z']='nan'
        return r
    converged = results.tables[0].data[5][1].strip()
    results = results.tables[1].data
    results = pd.DataFrame(results[1:], columns=['coef_id', 'coef', 'std err', 'z', 'P>|z|', '[0.025', '0.975]'])
    results['P>|z|'] = pvals
    results['z'] = tvals 
    results['coef'] = coeffs
    results['converged'] = converged
    results['pseudo_r2'] = pseudo_r2
    # results['adj_r2'] = adj_r2
    results["train_MSE"] = train_mse
    results["MSE"] = test_mse
    results['[0.025'] = cint_low
    results['0.975]'] = cint_high
    results["train_pearsonR"] = train_pear_R
    results["pearsonR"] = test_pear_R
    return results

In [23]:
regression_results = []
for d in drug_list[:1]:
    print(f'currently looking at drug {d}')
    c = 0
    xy = x.merge(y[y["drug_id"]==d], left_on='CellLine', right_on='cell_line_name')
    test_xy = x_test.merge(y[y["drug_id"] == d], left_on='CellLine', right_on='cell_line_name')
    for p in protein_list:

        
        data = xy[["ln_IC50", "max_screening_conc", p]].dropna()
        test_data = test_xy[["ln_IC50", "max_screening_conc", p]].dropna()

        # print(f'For drug {d} and {p} there are {len(data[p])} and {len(test_data[p])} cell lines')

        if len(test_data[p]) < 8:
            # print(f'Not enough test cell lines for protein {p} and drug {d}, n = {len(test_data[p])}')
            c += 1
            continue

        formula = "ln_IC50 ~ max_screening_conc + " + p
        ols = smf.ols(formula,data=data)
        results = ols.fit(disp=False, maxiter=1000)
        results = results_fit_to_df(results, ols, data["ln_IC50"].to_list(), test_data)
        results["drug"] = d
        results["n_train"] = len(data[p])
        results["n_test"] = len(test_data[p])
        regression_results.append(results.loc[[2]])
        del data, test_data   
    print(f'For {c} proteins we did not have enough data')
    del xy, test_xy
regression_results = pd.concat(regression_results)

currently looking at drug 1


C:\Users\wen068\AppData\Roaming\Python\Python311\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\wen068\AppData\Roaming\Python\Python311\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\wen068\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\wen068\AppData\Roaming\Python\Python311\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\wen068\AppData\Roaming\Python\Python311\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by

For 1326 proteins we did not have enough data


In [22]:
1.5*949/60

23.725

In [18]:
regression_results

,coef_id,coef,std err,z,P>|z|,[0.025,0.975],converged,pseudo_r2,train_MSE,MSE,train_pearsonR,pearsonR,drug,n_train,n_test
2,P37108SRP14HUMAN,0.344876,0.103,3.350153,0.000913,0.142280,0.547473,297,0.036651,1.028239,1.867878,0.191446,0.050281,1,297,79
2,Q96JP5ZFP91HUMAN,0.293598,0.130,2.263659,0.024617,0.037916,0.549281,212,0.023820,1.024769,2.191084,0.154336,-0.084071,1,212,63
2,Q9Y4H2IRS2HUMAN,-0.008152,0.200,-0.040702,0.967673,-0.409041,0.392738,60,0.000029,1.511382,1.972468,0.005344,-0.213663,1,60,19
2,P36578RL4HUMAN,-0.106366,0.124,-0.860458,0.390235,-0.349647,0.136914,297,0.002504,1.064687,1.881234,0.050035,-0.108920,1,297,79
2,Q6SPF0SAMD1HUMAN,0.108384,0.073,1.491971,0.136967,-0.034690,0.251458,252,0.008825,0.974873,1.740995,0.093943,0.140308,1,252,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,Q96RT7GCP6HUMAN,-0.009340,0.174,-0.053654,0.957555,-0.364363,0.345683,33,0.000093,0.604705,0.390081,0.009636,-0.155726,1,33,11
2,Q86SZ2TPC6BHUMAN,0.329737,0.213,1.547722,0.126331,-0.095391,0.754866,70,0.034028,1.068815,3.837595,0.184468,0.018681,1,70,17
2,Q5EBL4RIPL1HUMAN,0.013412,0.143,0.093868,0.925589,-0.273580,0.300404,52,0.000176,0.664233,0.957161,0.013274,-0.229536,1,52,17
2,Q5TA45INT11HUMAN,-0.119480,0.120,-0.997674,0.319944,-0.355992,0.117031,162,0.006183,1.185055,1.522771,0.078629,0.100663,1,162,37


In [45]:
regression_results.to_csv("protein_regression.csv", index=False)

pd.pivot_table(regression_results, values='pearsonR', index=['drug'], columns=['coef_id'], aggfunc=np.sum, fill_value=0).to_csv("protein_regression_pearsonR.csv")

,coef_id,coef,std err,z,P>|z|,[0.025,0.975],converged,pseudo_r2,adj_r2,train_MSE,MSE,train_pearsonR,pearsonR,drug
2,P37108SRP14HUMAN,0.344876,0.103,3.350153,0.000913,0.142280,0.547473,297,3.665141e-02,0.033386,1.028239,1.867878,0.191446,0.050281,1
2,Q96JP5ZFP91HUMAN,0.083934,0.035,2.367290,0.018564,0.014156,0.153712,297,1.864266e-02,0.015316,1.047461,1.920604,0.136538,-0.077095,1
2,Q9Y4H2IRS2HUMAN,-0.048480,0.055,-0.883640,0.377612,-0.156455,0.059496,297,2.648817e-03,-0.004136,1.064532,1.881244,0.051467,-0.056817,1
2,P36578RL4HUMAN,-0.106366,0.124,-0.860458,0.390235,-0.349647,0.136914,297,2.503509e-03,-0.000878,1.064687,1.881234,0.050035,-0.108920,1
2,Q6SPF0SAMD1HUMAN,0.083535,0.040,2.105337,0.036107,0.005448,0.161623,297,1.480281e-02,0.011463,1.051559,1.857387,0.121667,0.086719,1
2,O76031CLPXHUMAN,-0.272634,0.136,-2.011889,0.045141,-0.539325,-0.005942,297,1.353529e-02,0.010191,1.052912,1.926358,0.116341,-0.131889,1
2,Q8WUQ7CATINHUMAN,0.167690,0.244,0.687954,0.492024,-0.312029,0.647409,297,1.607213e-03,-0.005185,1.065644,1.859011,0.040090,0.109621,1
2,A6NIH7U119BHUMAN,0.064529,0.045,1.427413,0.154519,-0.024440,0.153498,297,6.859431e-03,0.003493,1.060038,1.860427,0.082822,0.071958,1
2,Q9BTD8RBM42HUMAN,0.057629,0.071,0.807258,0.420168,-0.082866,0.198123,297,2.204167e-03,-0.001178,1.065006,1.871115,0.046949,-0.024887,1
2,Q9P258RCC2HUMAN,0.129715,0.057,2.260317,0.024530,0.016773,0.242657,297,1.702392e-02,0.013692,1.049188,1.812365,0.130476,0.224306,1
